In [38]:
cd ../src

/Users/dani/Google Drive/Uni/Master/P4 2021/DD2424 - Deep Learning for Data Science/Project/ScarceClassifier/src


In [39]:
from d02_data.load_data import get_dataloaders_ssl
from d04_mixmatch.wideresnet import WideResNet
import torch
import torch.nn as nn
import numpy as np
import time
import yaml
from pathlib import Path


In [40]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cpu


In [41]:
configuration = yaml.load(Path("config.yml").read_text(), Loader=yaml.SafeLoader)

config = configuration['MixMatchTrainer']
params = configuration['WideResNet']
adam = config['adam']
sgd = config['sgd']

batch_size = configuration['cifar_10']['batch_size']
num_labeled = configuration['cifar_10']['num_labeled']
n_steps = 1000
K = config['K']
lambda_u_max = config['lambda_u_max']
steps_validation = config['steps_validation']
steps_checkpoint = config['steps_checkpoint']
step_top_up = 500
optimizer = config['optimizer']
adam_params = adam['lr'], adam['weight_decay']
sgd_params = sgd['lr'], sgd['momentum'], sgd['weight_decay']

wideresnet_params = (params['depth'], params['k'], params['n_out'])

In [42]:
# Get data
_, _, _, test_loader = get_dataloaders_ssl(path='../data', batch_size=batch_size, num_labeled=num_labeled)


Files already downloaded and verified
Files already downloaded and verified


In [43]:
model = WideResNet(depth=28, k=2, n_out=10).to(device)

In [47]:
checkpoint = torch.load('../models/checkpoint_30k.pt', map_location=torch.device('cpu'))


In [48]:
checkpoint.keys()


dict_keys(['step', 'model_state_dict', 'optimizer_state_dict'])

In [49]:
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [50]:
def evaluate(dataloader):
    criterion = nn.CrossEntropyLoss()
    model.eval()
    correct, total, loss = 0, 0, 0
    with torch.no_grad():
        for i, data in enumerate(dataloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss += criterion(outputs, labels).item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        loss /= dataloader.__len__()

    acc = correct / total * 100
    return loss, acc

In [51]:
test_val, test_acc = evaluate(test_loader)
print("Training done!!\t Test loss: %.3f \t Test accuracy: %.3f" % (test_val, test_acc))

Training done!!	 Test loss: 2.445 	 Test accuracy: 32.160
